<a href="https://colab.research.google.com/github/Qualot/deep-learning-from-scratch/blob/colab/ch04/TwoLayeresNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [7]:
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c) #prevents overflow
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y

In [ ]:
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [8]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #initialize weights and biasses
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

  #x: input data, t: teacher data
  def loss(self, x, t):
    y = self.predict(x)

    return cross_entropy_error(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  #x: input, t: teacher
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

#Testing part

In [16]:
a = np.random.randn(10, 5)
print(a)
b = np.random.randn(10, 5)
print(b)

a = np.argmax(a, axis=1)
b = np.argmax(b, axis=1)
print(a)
print(b)
print(a==b)

[[ 1.55398611 -1.5625352   0.26405701  0.78497733 -2.02504349]
 [-0.63353341 -0.73216817  0.4111785   1.23773409 -0.55341871]
 [-0.97541038  1.47238285  1.73105479 -1.19446728  1.31029978]
 [-0.13764955  0.14971464 -1.6176078  -0.34585941 -1.97763972]
 [ 1.82569332  1.19885623 -0.66927943  0.91057178 -1.83706886]
 [-0.91500902  1.66269707 -0.03807004  2.03979789  0.70442019]
 [ 0.24414984  1.89347267  1.48252402 -0.22732093 -0.75443784]
 [ 1.65046615  0.65332137  0.12591594 -0.96367816 -0.4955121 ]
 [ 1.23195819  1.52342052 -1.76065933  0.2908632  -0.11990235]
 [ 1.0933422  -0.12469022 -1.51488744 -1.10489281  2.59651427]]
[[-0.03108484  0.48559924 -0.21366483 -2.51778696 -0.0380181 ]
 [-1.38551227 -0.36436333  1.14223142  1.26000476  1.41616356]
 [ 1.03651942  0.17282124 -0.96426174 -0.97540925  0.34985807]
 [-0.90616789  0.07773618 -0.03042346  0.55926871  1.27759579]
 [-0.99259322  0.14153174  0.79428487  0.20330144  2.79248668]
 [-0.99645494 -0.80088313  1.29916789  0.30599348 -0.6